In [ ]:
!pip install datasets transformers sentencepiece
!pip install huggingface_hub
!pip install accelerate -U
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4

In [ ]:
import os
!pip install huggingface_hub

!apt install git-lfs
!git lfs install

from huggingface_hub import notebook_login
notebook_login()

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Git LFS initialized.


In [ ]:
import os
import json
import torch
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import RobertaTokenizerFast, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import load_dataset

from tqdm.notebook import tqdm
from IPython.display import HTML
from functools import partial
from difflib import SequenceMatcher

pd.set_option("display.max_columns", None)

dataset_directory = '/content/drive/MyDrive/Codehealer/'

os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
Codehealer = load_dataset("json", data_files={"train": dataset_directory+"Codehealer_train.json", "test": dataset_directory+"Codehealer_test.json"}, field='data')

filtered_train_dataset = Codehealer["train"].filter(lambda code_sample: code_sample["returncode"] != 0).train_test_split(test_size=0.1)
filtered_test_dataset = Codehealer["test"]

In [ ]:
import torch
from transformers import RobertaTokenizerFast
from functools import partial

max_input_length = 256
max_output_length = 512

def process_and_tokenize_samples(tokenizer, code_sample):
    tokenized_inputs = tokenizer(text=code_sample["error_classification"], text_pair=code_sample["original_code"], max_length=max_input_length, padding=True, truncation=True)
    tokenized_targets = tokenizer(code_sample["modified_code"], max_length=max_output_length, padding=True, truncation=True)

    label_ids = tokenized_targets.input_ids
    label_ids = torch.tensor(label_ids)
    label_ids[label_ids == tokenizer.pad_token_id] = -100

    tokenized_inputs["labels"] = label_ids.tolist()
    return tokenized_inputs

tokenizer_instance = RobertaTokenizerFast.from_pretrained("Salesforce/codet5-base")

tokenized_train_dataset = filtered_train_dataset.map(partial(process_and_tokenize_samples, tokenizer_instance), batched=True, batch_size=4, remove_columns=filtered_train_dataset["train"].column_names)
tokenized_test_dataset = filtered_test_dataset.map(partial(process_and_tokenize_samples, tokenizer_instance), batched=True, batch_size=4, remove_columns=filtered_test_dataset.column_names)


In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, DataCollatorForSeq2Seq
import os

output_directory_path = '/content/codet5-base-bug-fix-outputs'
logging_directory_path = '/content/codet5-base-logs'

os.makedirs(output_directory_path, exist_ok=True)
os.makedirs(logging_directory_path, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_directory_path,
    num_train_epochs=1,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logging_directory_path,
    logging_steps=1_000,
    save_strategy="no",
    push_to_hub=False,
)

model_instance = T5ForConditionalGeneration.from_pretrained("Salesforce/codet5-base")

data_collator_instance = DataCollatorForSeq2Seq(tokenizer_instance, model_instance, padding=True)

In [ ]:
def evaluate_model(predictions_and_labels):
    predictions, labels = predictions_and_labels

    predictions = np.argmax(predictions[0], axis=2)

    flattened_predictions = [pred for sequence in predictions for pred in sequence]
    flattened_labels = [label for sequence in labels for label in sequence]

    return {
        "precision": precision_score(flattened_labels, flattened_predictions, average="weighted"),
        "recall": recall_score(flattened_labels, flattened_predictions, average="weighted"),
        "f1": f1_score(flattened_labels, flattened_predictions, average="weighted"),
        "accuracy": accuracy_score(flattened_labels, flattened_predictions)
    }

In [ ]:
training_instance = Trainer(
    model=model_instance,
    args=training_args,
    train_dataset=tokenized_train_dataset["train"],
    eval_dataset=tokenized_train_dataset["test"].select(range(10)),
    tokenizer=tokenizer_instance,
    compute_metrics=evaluate_model,
    data_collator=data_collator_instance,
)

training_instance.train()

In [ ]:
training_instance.evaluate()

In [ ]:
saved_model_directory_path = '/content/drive/MyDrive/saved_model_epoch'

os.makedirs(saved_model_directory_path, exist_ok=True)

In [ ]:
model_instance.save_pretrained(saved_model_directory_path)

In [ ]:
model_instance = T5ForConditionalGeneration.from_pretrained(saved_model_directory_path)

In [ ]:
def generate_code_suggestions(tokenizer_instance, model_instance, error_message, original_code_snippet, beam_search_size=5):
    tokenized_inputs = tokenizer_instance(text=error_message, text_pair=original_code_snippet, max_length=512, padding=True, truncation=True, return_tensors="pt").to(model_instance.device)
    generated_code_ids = model_instance.generate(num_beams=beam_search_size, no_repeat_ngram_size=2, num_return_sequences=beam_search_size, max_length=512, **tokenized_inputs).cpu().detach().numpy()

    return tokenizer_instance.batch_decode(generated_code_ids, skip_special_tokens=True)

In [ ]:
error_message_example = "SyntaxError: Missing parentheses in call to 'print'. Did you mean print(l[i])?"
code_snippet_example = "l=[]\nwhile True:\n    try:\n        l.append(input())\n    except SyntaxError:\n        break\nl.sort(reverse = True)\nfor i in range(3):\n    print l[i]"

tokenized_inputs_example = tokenizer_instance(text=error_message_example, text_pair=code_snippet_example, max_length=512, padding=True, truncation=True)

generated_code_example = generate_code_suggestions(tokenizer_instance, model_instance, [error_message_example], [code_snippet_example], beam_search_size=5)[0]

print("Original Code:")
print(code_snippet_example)
print("Suggested Fix:")
print(generated_code_example)
